In [101]:
import os

In [102]:
%pwd

'C:\\Users\\vishnu k k\\Chicken-Disease-Classification--Project'

In [103]:
os.chdir("../")

In [104]:
%pwd


'C:\\Users\\vishnu k k'

In [105]:
os.chdir(r"C:\\Users\\vishnu k k\\Chicken-Disease-Classification--Project")

In [106]:
%pwd

'C:\\Users\\vishnu k k\\Chicken-Disease-Classification--Project'

In [107]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [108]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [109]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

    

In [110]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size


In [115]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {os.path.getsize(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
    
        
            

In [116]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    
    

[2024-01-06 16:24:14,525: INFO: common: yaml file:config\config.yaml loaded successfully]
[2024-01-06 16:24:14,527: INFO: common: yaml file:params.yaml loaded successfully]
[2024-01-06 16:24:14,529: INFO: common: Created directory at: artifacts]
[2024-01-06 16:24:14,531: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-01-06 16:24:14,531: INFO: 47205051: File already exists of size: 11616915]
